In [28]:
#from bs4 import BeautifulSoup
import gcsfs  ## Google Cloud Storage file system
import os
import pandas as pd
import requests
import tarfile
import wfdb
import matplotlib.pyplot as plt
import numpy as np
import datetime


In [29]:
from src.data.parsing import parse_txt

from src.data.gcp_pull_waveform_data import generate_record_map,generate_waveform_dataset

from src.data.bp_features import avg_bp,clean_bp_summary,merge_df

In [30]:
%env PROJECT_ID=qwiklabs-gcp-04-133e595cc3fe

env: PROJECT_ID=qwiklabs-gcp-04-133e595cc3fe


In [31]:
fs = gcsfs.GCSFileSystem(project=os.environ['PROJECT_ID'])
bucket = 'physionet_2009'
fs.ls(bucket)

['physionet_2009/mimic2cdb',
 'physionet_2009/processed',
 'physionet_2009/test-set-a',
 'physionet_2009/test-set-b',
 'physionet_2009/train_wave']

In [32]:
# Get Train data Id

In [33]:
training_set = [
's21775', 's20658', 's22466','s05336', 's06349',
's08718', 's20794', 's24799', 's26318', 's14204',
's25699', 's07125', 's19208', 's12821', 's06637',
    
's02395', 's08779', 's23641', 's24924', 's00439', 
's23015', 's19603', 's02172', 's26105', 's23594',
's21817', 's24984', 's25602', 's23591', 's15687',
    
's17765', 's04860', 's26097', 's00318', 's14495',
's22888', 's26296', 's06180', 's07468', 's24004',
's03133', 's02280', 's09672', 's19418', 's15465',
    
's05786', 's25222', 's02561', 's24923', 's16019',
's04286', 's17069', 's07860', 's23020', 's24431',
's22657', 's09341', 's05126', 's10611', 's05289'
]

training_labels = [
'H1', 'H1', 'H1', 'H1', 'H1', 'H1', 'H1', 'H1',
'H1', 'H1', 'H1', 'H1', 'H1', 'H1', 'H1',
    
'H2', 'H2', 'H2', 'H2', 'H2', 'H2', 'H2', 'H2',
'H2', 'H2', 'H2', 'H2', 'H2', 'H2', 'H2',
    
'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 
'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 'C1',

'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2',
'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2'
]

len(training_set) == len(training_labels)

True

In [34]:
# Pull in records

In [35]:
record_map = generate_record_map(bucket, fs, training_set)

Dimensions of data set: (0, 6)
Data set reflects data for 0 clinical IDs
Data set reflects data for 0 waveform IDs


In [15]:
# turn into useful data frame

In [21]:
x = {e:generate_waveform_dataset(e, record_map) for e in record_map['waveform_entities'][0:2]}

In [27]:
# calculate buckets
x

{}

In [26]:
#avg_bp(x['a40802'], time_chunk = 60 * 5, time_window = 60,waveform_type = 'ABP')

bp_summary_outcome = {e: avg_bp(x[e], time_chunk = 60 * 5, time_window = 60,waveform_type = 'ABP') for e in x.keys()}
bp_summary_outcome = {e: clean_bp_summary(x) for e, x in bp_summary_outcome.items()}
bp_summary_outcome

{}

In [24]:
bp_summary_inputs = {e: avg_bp(x[e], time_chunk = 60 * 20, time_window = 60,waveform_type = 'ABP') for e in x.keys()}
bp_summary_inputs = {e: clean_bp_summary(x) for e, x in bp_summary_inputs.items()}

In [25]:
final_df = pd.concat([merge_df(e_in,e_out,df_in,df_out) for (e_in, df_in),(e_out, df_out) in zip(bp_summary_inputs.items(), bp_summary_outcome.items())])

ValueError: No objects to concatenate

In [ ]:
final_df
#df_out = bp_summary_outcome['a40802'] 
#df_in = bp_summary_inputs['a40802']

In [ ]:
#final_df = df_out.merge(right = df_in, on = 'start_window', suffixes = ['_outputs','_inputs'])